### Fetch the reserves of the Uniswap V2 pool

In [3]:
from web3 import Web3
import json

# Connect to a node. If using Infura, it should look like this:
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/<YOUR_INFURA_PROJECT_ID>'))

# Load the ABI of the UniswapV2Pair contract
with open('UniswapV2Pair.json', 'r') as f:
    pairABI = json.load(f)

# Create a contract object with the pair address.
weth_usdt_addr = '0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852'
pair = w3.eth.contract(address=weth_usdt_addr, abi=pairABI)

# Call the getReserves() function
reserves = pair.functions.getReserves().call()

# Print the reserves: [res_weth, res_usdt, timestamp]
print(reserves) # [16955718197081157997253, 29720979785430, 1686648623]

HTTPError: 401 Client Error: Unauthorized for url: https://mainnet.infura.io/v3/%3CYOUR_INFURA_PROJECT_ID%3E

### Fetch the decimals of the USDT token

In [4]:
# Load the ABI of the ERC20 contract
with open('ERC20ABI.json', 'r') as f:
    ercABI = json.load(f)

# Create a contract object with the token address. Here we used USDT.
token = w3.eth.contract(address='0xdAC17F958D2ee523a2206206994597C13D831ec7', abi=ercABI)

# Call the decimals() function
decimals = token.functions.decimals().call()

# Print the decimals
print(decimals) # 6

HTTPError: 401 Client Error: Unauthorized for url: https://mainnet.infura.io/v3/%3CYOUR_INFURA_PROJECT_ID%3E

### Upload the TestSwap.sol contract on the local Anvil node

In [5]:
# Code to upload the TestContract to a local node without using Remix
contractContent = """
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0; // Any Solidity 0.8.x version

interface IUniswapV2Pair { // Using interfaces has the same role as using ABIs in web3.py
    function swap(uint amount0Out, uint amount1Out, address to, bytes calldata data) external;
}

interface IWETH { // To simply the code, we only include the functions we use in the interface declaration
    function deposit() external payable;
}

interface IERC20 {
    function transfer(address recipient, uint256 amount) external returns (bool);
}

contract TestSwap {
    // Store the addresses used as constants for readability. This does not cost any gas.
    address constant USDT = 0xdAC17F958D2ee523a2206206994597C13D831ec7;
    address constant WETH = 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2;
    address constant UNI_PAIR = 0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852;

    // Define the function the will be called by web3.py
    function startSwap(uint usdtOut) external payable {// External specifies that this function is called from outside this contract. The payable keyword makes the function capable of receive ETH when called.
        // Wrap the ETH received in the transaction into WETH. The amount of ETH received is available in msg.value
        // We call the WETH contract's deposit() function and forward the same amount of ETH we received in the transaction
        IWETH(WETH).deposit{value: msg.value}(); // The {value: msg.value} syntax is used to forward ETH to a contract when calling it. The parameters are still between parentheses (here there are none).

        // Now that we have WETH, we can send it to the Uniswap Pair contract.
        IERC20(WETH).transfer(UNI_PAIR, msg.value); // We use the ERC20 transfer() function. The amount of WETH is the same as the ETH received.

        // Just like with the WETH contract, we enclose the address with the interface we want to use, and call the function of interest.
        // Remember that USDT is the token1 of the pair. We don't want to swap out any ETH, so we pass 0 as the first parameter.
        // To specify the current contract as the recipient of the output, we could have used address(this) instead of msg.sender. Here we sent the output to the address that called the current transaction.
        IUniswapV2Pair(UNI_PAIR).swap(0, usdtOut, msg.sender, new bytes(0)); // The last parameter is the data parameter, which we don't use here. We pass an empty bytes array.
    }
}
"""

import solcx
version = "0.8.0"
filename = "TestSwap.sol"
Output = "ir"

compiled_sol = solcx.compile_standard(
    {
        "language": "Solidity",
        "sources": {filename: {"content": contractContent}},
        "settings": {
            "outputSelection": {
                "*": {"*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap",Output]}
            }
        },
    },
    solc_version=version,
)

name = filename.split('.')[0]
res_bytecode = compiled_sol["contracts"][filename][name]["evm"]["bytecode"]["object"]
res_abi = compiled_sol["contracts"][filename][name]["abi"]
# print(compiled_sol["contracts"][filename][name][Output])

def uploadContract(addr, privateKey, bytecode, abi, provider, chainId=1, gasPrice=50*10**9): 
    """Deploy the provided bytecode to the blockchain"""

    print(f"Uploading smart contract from addr: {addr[:8]}...")
    #convert this into a ethereum contract object
    _contract = provider.eth.contract(abi=abi, bytecode=bytecode)
    #Get the nonce
    nonce = provider.eth.get_transaction_count(addr)
    #Define tx data
    txdata = {"chainId": chainId, "gas": 5000000, "gasPrice": gasPrice, "nonce": nonce}
    #Build Transaction
    transaction = _contract.constructor().build_transaction(txdata)
    #Sign the transaction
    signed = provider.eth.account.sign_transaction(transaction, private_key=privateKey)
    #Send the transaction
    tx_hash = provider.eth.send_raw_transaction(signed.rawTransaction)

    #Check for receipt
    receipt = provider.eth.wait_for_transaction_receipt(tx_hash)
    print(f'Transaction status: {receipt["status"]}')
    print(f'Gas used by Tx: {receipt["gasUsed"]}')
    print(f'Contract uploaded at {receipt.contractAddress}')
    contract_address = receipt.contractAddress
    full_contract = provider.eth.contract(address=contract_address,abi=abi)

    return (full_contract, contract_address)

# Anvil
ACC_PK = "0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80"
ACC_ADDR = "0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266"

# # Ganache
# ACC_PK = "0x316d944ace887053df5243fce6fd7f9cb2e3d50e9446204647abec1afc480ada"
# ACC_ADDR = "0x69AEFc283fA06c713EBfa2DeE98A0D9826BF29BC"

w3local = Web3(Web3.HTTPProvider("http://localhost:8545"))
res_contract, res_address = uploadContract(ACC_ADDR, ACC_PK, res_bytecode, res_abi, w3local)

Uploading smart contract from addr: 0xf39Fd6...
Transaction status: 1
Gas used by Tx: 321231
Contract uploaded at 0x3abBB0D6ad848d64c8956edC9Bf6f18aC22E1485


### Perform a swap by calling the TestSwap.sol contract

In [6]:
from eth_account import Account

##### Parameters/constants of the script #####
# Url of the local Anvil node
NODE_URL = "http://localhost:8545"

# Private key of the account that will sign the transaction.
# Anvil gives private keys of test accounts that have a lot of ETH in them already.
# This example PK probably has 10000 ETH in it.
# Anvil
SENDER_PK = "0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80"
# Ganache
# SENDER_PK = "0x316d944ace887053df5243fce6fd7f9cb2e3d50e9446204647abec1afc480ada"

# Address of the WETH contract. Don't forget to use checksum addresses (Have capital letters in them. You can use Web3.toChecksumAddress() to convert an address to checksum format, or copy it from EtherScan)
WETH_ADDRESS = "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2"
# USDT ERC20 contract address
USDT_ADDRESS = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
# Address of the Uniswap WETH-USDT pair
UNI_PAIR = "0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852"

# Input amount of ETH to swap. 1 ETH = 10^18 wei
ETH_INPUT = 10**18 # Python returns an integer. Careful not to use floats, as will be the case with other languages.
# Fee taken by Uniswap
UNI_FEE = 0.003 # 0.3%

# TestSwap contract address, as given by Remix after deployment, or in the Anvil console log.
TESTSWAP_ADDRESS = res_address
##############################################

# Connect to the local Anvil node
w3 = Web3(Web3.HTTPProvider(NODE_URL))

# The account used to sign the transaction. Created from the private key given. Anvil gives private keys of test accounts that have a lot of ETH.
signer = Account.from_key(SENDER_PK)
address = signer.address # Calculate the address from the private key
print("Using address:", address)

# First we will find the reserves of the Uniswap pair contract. We need to do this because we need to know how much USDT we will get for the ETH we send.
# Load the ABI. You can save it as a JSON file, or you can copy it directly, as we do here. Note that we use an incomplete ABI, as only need the getReserves() function.
# pairABI = json.load(open("UniswapV2Pair.json", "r"))
pairABI = [
    {
        "constant": True,
        "inputs": [],
        "name": "getReserves",
        "outputs": [
            {
                "internalType": "uint112",
                "name": "_reserve0",
                "type": "uint112"
            },
            {
                "internalType": "uint112",
                "name": "_reserve1",
                "type": "uint112"
            },
            {
                "internalType": "uint32",
                "name": "_blockTimestampLast",
                "type": "uint32"
            }
        ],
        "payable": True,
        "stateMutability": "view",
        "type": "function"
    }
]

# Create a contract object for the Uniswap pair contract.
pairContract = w3.eth.contract(address=UNI_PAIR, abi=pairABI)

# Call the getReserves() function of the Uniswap pair contract.
pairReserves = pairContract.functions.getReserves().call()

# Use the formula (1) to calculate dy, the amount of USDT we will get for the ETH we send.
# Remember that Uniswap V2 takes a 0.3% fee on the input amount.
# dy = y * (x/(x + dx) - 1) (1)
x = pairReserves[0] # x is the amount of WETH in the pair
y = pairReserves[1] # y is the amount of USDT in the pair
dx = ETH_INPUT * (1 - UNI_FEE) # dx is the amount of WETH that will be used by the constant product formula
usdtOutput = y * (1 - x/(x + dx)) # dy is the amount of USDT we will get for the ETH we send
usdtOutput = int(usdtOutput) # Truncate into integer, as the formula returns a float and we must never overestimate the amount, as it would make the swap fail. We use abs() to make the number positive as the formula returns the reserve difference for the pool.
print("Expected USDT output:", usdtOutput) # 1747251875

# Get the ABI of the TestSwap contract. The ABI can be copied from Remix.
# contractABI = json.load(open("TestSwapABI.json", "r")
contractABI = [
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "usdtOut",
				"type": "uint256"
			}
		],
		"name": "startSwap",
		"outputs": [],
		"stateMutability": "payable",
		"type": "function"
	}
]

# Create a contract object from the ABI and address of the TestSwap contract. The ABI can be copied from Remix.
testswap = w3.eth.contract(address=TESTSWAP_ADDRESS, abi=contractABI)

# Create a transaction object from the function we want to call and its parameters.
# Here we call the startSwap() function, and pass the output usdt amount we calculated above as a parameter.
txn = testswap.functions.startSwap(usdtOutput).build_transaction({
    'chainId': 1, # Chain ID of the Node. Ganache uses 1337, Anvil uses 1.
    'gas': 500000, # Gas limit.
    'gasPrice': w3.eth.gas_price, # Gas price. We use the gas price of the node.
    'nonce': w3.eth.get_transaction_count(address), # Nonce is the number of transactions sent from the account. It is used to prevent replay attacks.
    'value': ETH_INPUT # The amount of ETH we send to the contract. We use the same amount as the amount of ETH we want to swap.
})

# Sign the transaction with the private key of the account.
signed_txn = w3.eth.account.sign_transaction(txn, private_key=SENDER_PK)

# Send the transaction to the node.
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print("Tx hash:", tx_hash.hex()) # Tx hash: 0xe06a00d0bcda54e2e3d79900a4629d5f9fe7e5a6a7382db994290d09a68d0017
print("Waiting for transaction to be mined...")
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("Transaction mined in block", tx_receipt.blockNumber) # Transaction mined in block 17470539
print("Transaction status:", tx_receipt.status, "(Success)" if tx_receipt.status == 1 else "(Failure)") # Transaction status: 1 (Success)
print("Gas used:", tx_receipt.gasUsed) # Gas used: 124047

Using address: 0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266
Expected USDT output: 1747046127
Tx hash: 0x406b7fb7ea540e378dfe620dfbb318788bfd15ed6121a1fad641a71bf9104582
Waiting for transaction to be mined...
Transaction mined in block 17470541
Transaction status: 1 (Success)
Gas used: 106947


### 